In [76]:
import numpy as np
import matplotlib.pyplot as plt
from sympy import *
from numpy.linalg import norm
import regression as reg
from data import funcs 
import template as example

In [77]:
# TODO: init_point -- List[n, 1]
# TODO: inside -- as vector
def bfgs(f, grad, init_point, eps=1e-4, max_iter=100):
    point = init_point
    H = np.eye(len(point))
    E = np.eye(len(point))
    
    a = Symbol('a')
    
    for it in range(max_iter):
        g = grad(point)
        p = -H.dot(g)
        
        print("g=", norm(g))
        print("p=", p)
        print("H=", H)
        if norm(g) < eps:
            break
        
        sol = solve(simplify(f(a * p + point)).diff(), a)
    
        print(simplify(f(a * p + point)))
        alpha = float(sol[0])
            
        new_point = point + alpha * p
        new_grad = grad(new_point)
        
        s = alpha * p
        y = new_grad - g
        
        rho = 1 / (y.transpose().dot(s))
        
        H = (E - rho * s.dot(y.transpose())).dot(H).dot(E - rho * y.dot(s.transpose())) + rho * s.dot(s.transpose())
        point += s
            
    return point

In [78]:
def f(point):
    x, y = point[0][0], point[1][0]
    return x ** 2 - x * y + y ** 2 + 9 * x - 6 * y + 20

def grad(point):
    x, y = point[0][0], point[1][0]
    return np.array([[2 * x - y + 9], [-x + 2 * y - 6]])

In [79]:
def create_test(func, jacobian, regression, grad):
    def run(x: np.ndarray, y: np.ndarray, st: np.ndarray, eps: float, max_iter: int):
        r = regression(func, x, y)
        F = lambda betta: sum(r(betta) ** 2)
        return bfgs(F, grad(x, jacobian, r), st, eps, max_iter)

    return run

In [80]:
x0 = np.array([[0.], [2.]])
print(x0)
epsilon = 0.001

start = 0
end = 10
steps = 50
test_x, test_y = example.generate_test(np.sin, start, end, steps)

func, jac = funcs["pol"]
test = create_test(func, jac, reg.regression, reg.grad)

betta = bfgs(f, grad, x0)
print(betta)
# X = np.linspace(start, end, steps)
# Y = f(X)(betta)

# example.show(test_x, test_y, X, Y)

[[0.]
 [2.]]
g= 7.280109889280518
p= [[-7.]
 [ 2.]]
H= [[1. 0.]
 [0. 1.]]
67.0*a**2 - 53.0*a + 12.0
g= 2.4448130225195768
p= [[-1.46107151]
 [-2.12519492]]
H= [[0.72282245 0.41501448]
 [0.41501448 0.78547561]]
3.54612165558928*a**2 - 5.97711071508131*a + 1.51865671641791
g= 0.0
p= [[-0.]
 [-0.]]
H= [[0.66666667 0.33333333]
 [0.33333333 0.66666667]]
[[-4.]
 [ 1.]]
